In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OpenVINO 준비

In [15]:
%pip install -q "openvino>=2023.1.0"

In [16]:
!pip install openvino-dev

keras .tf 모델을 Intel OpenVINO IR 파일(xml, bin)로 한다.

In [17]:
import openvino
import openvino as ov
from openvino.tools.mo import convert_model

BASE_PATH = "/content/drive/MyDrive/data/"
ov_model = convert_model(BASE_PATH + 'efficientnet-b0-epoch20.onnx')
ov.save_model(ov_model, '/content/drive/MyDrive/data/efficientnet-b0.xml')

### 테스트 파일 목록으로 읽어와서 답안 출력하기.



In [18]:
# --------------------------- Step 1. Initialize OpenVINO Runtime Core ------------------------------------------------
core = ov.Core()

In [19]:
# --------------------------- Step 2. Read a model --------------------------------------------------------------------
# (.xml and .bin files) or (.onnx file)
model = core.read_model(BASE_PATH+'efficientnet-b0.xml')

if len(model.inputs) != 1: print('Sample supports only single input topologies')
if len(model.outputs) != 1: print('Sample supports only single output topologies')

In [20]:
import os

directory_path = "/content/drive/MyDrive/data/test"

# Get a list of all files in the directory
file_list = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Print the list of files
print("List of files in the directory:")
for file in file_list:
    print(file)

List of files in the directory:
137492.jpg
0576Qr.jpg
03T7NT.jpg
17yo_A.jpg
1G0_Z9.jpg
0m_wUL.jpg
0KHL57.jpg
0_v_9B.jpg
126Ek_.jpg
0N8A9_.jpg
0WS5Sa.jpg
0v40Gb.jpg
1evyt_.jpg
0e_6x3.jpg
0__4_v.jpg
046w29.jpg
0e_sET.jpg
0ScHiO.jpg
1gmS4_.jpg
0_0_02.jpg
0qUaOB.jpg
1eo1_W.jpg
1976ku.jpg
19_88_.jpg
190E3d.jpg
0O2x31.jpg
1f4WOP.jpg
0_zDfn.jpg
17_4M8.jpg
0PJb6K.jpg
0tTcFg.jpg
0d2E8g.jpg
0S_ZY_.jpg
13TXK9.jpg
02x_DZ.jpg
0ojCP8.jpg
079hEu.jpg
0i1vdc.jpg
123_pA.jpg
02L38o.jpg
0SH0ge.jpg
0B02VC.jpg
2J8i44.jpg
31mqg1.jpg
2MgE_G.jpg
370sa_.jpg
1ZG6nY.jpg
2_L2iP.jpg
3990uB.jpg
2q6F_0.jpg
2uOJ1A.jpg
1ss5_T.jpg
2_h_Ij.jpg
3c3f2e.jpg
2pQp_R.jpg
3GVu__.jpg
30Npo6.jpg
2l6dPa.jpg
26Oq_r.jpg
2_1a2F.jpg
1o6p9_.jpg
2e9fEJ.jpg
2H910Q.jpg
1iT4_t.jpg
2ZRF6q.jpg
2vO1ZL.jpg
3Bev14.jpg
2HLn7H.jpg
2eLPY4.jpg
222749.jpg
25thI8.jpg
1yEJ3J.jpg
2Lu9K_.jpg
3BEJ4_.jpg
2g5W__.jpg
30vwgn.jpg
2gA7M4.jpg
2r622c.jpg
281FGu.jpg
2W0n0t.jpg
1ZJ90p.jpg
31dVZK.jpg
23a5__.jpg
2gmZnB.jpg
1kZL_P.jpg
2_1Sfm.jpg
1RasnU.jpg
2P2j1U.jpg


In [21]:
len(file_list)

967

In [22]:
import pickle

with open(BASE_PATH+'/class4.pickle', 'rb') as f:
    labels = pickle.load(f)

파일 추론하기

In [23]:
import pandas as pd
import cv2
import numpy as np

import torch # 추가한 코드 team.grinder
import torch.nn.functional as F # 추가한 코드 team.grinder




# --------------------------- Step 4. Loading model to the device -----------------------------------------------------
print('Loading the model to the plugin')
compiled_model = core.compile_model(model, 'CPU')
img_height = 224

result = []

res = open(BASE_PATH+"result.txt", "w")

print('Starting inference in synchronous mode')
cnt = 0;

for file in file_list:
    # --------------------------- Step 5. Set up input --------------------------------------------------------------------
    # Read input image
    image_path = directory_path + "/" + file
    org_image = cv2.imread(image_path)
    image = cv2.resize(org_image, (img_height,img_height), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    #img = cv2.resize(img, (224,224), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.

    # Add N dimension
    nchw_tensor = np.expand_dims(image, 0)

    # NHWC -> NCHW
    input_tensor = np.transpose(nchw_tensor, (0,3,1,2)) #(0,1,2,3)) <- 형식을 위해 Grinder팀 내에서 변경


    # --------------------------- Step 6. Create infer request and do inference synchronously -----------------------------
    results = compiled_model.infer_new_request({0:input_tensor})

    # --------------------------- Step 7. Process output ------------------------------------------------------------------
    predictions = next(iter(results.values()))
    output_node_name = next(iter(results.keys())) #
    #for (k,v) in results.items():
    #  print(k,'=',v)

    #print("values: ",end=' ')
    #print(results.values())

    # Change a shape of a numpy.ndarray with results to get another one with one dimension

    # ------ 추가한 코드 # 추가한 코드 team.grinder ------ #
    # probs = predictions.reshape(-1)

    predictions_tensor = torch.from_numpy(predictions) # NumPy 배열인 predictions을 PyTorch 텐서로 변환
    probs_tensor = F.softmax(predictions_tensor, dim=1) # PyTorch 텐서에 softmax 적용
    probs = probs_tensor.numpy().reshape(-1) # 다시 NumPy 배열로 변환 (추후 사용을 위해)
    # ----------------------------------------------- #

    #print(outs)
    #print("probs: ",end=' ')
    #print(probs)

    # Get an array of 4 class IDs in descending order of probability
    top_4 = np.argsort(probs)[-4:][::-1]
    print(top_4)

    #res = res[output_node_name]
    #idx = np.argsort(res[0])[-1]
    #prob = res[0][idx]*100

    header = 'class_id            probability'
    print(f'Image path: {image_path}')
    print('Top 4 results: ')
    print(header)
    print('-' * len(header))

    for class_id in top_4:
        probability_indent = ' ' * (len('class_id           ') - len(labels[class_id]) + 1)
        print(f'{labels[class_id]}{probability_indent}{probs[class_id]*100:.7f}')

    #print(labels[top_4[0]], probs[top_4[0]]*100)
    #print(labels[top_4[1]], probs[top_4[1]]*100)
    #result.append(top_4[0])
    #res.write(file+':'+str(labels[top_4[0]])+'\n')
    res.write(file+':top1 = ('+labels[top_4[0]]+')'+str(probs[top_4[0]]*100)+', top2 = ('+labels[top_4[1]]+')'+str(probs[top_4[1]]*100)+'\n')

#res.write(str(result)+'\n')
res.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
-------------------------------
NoOsteoarthritis    97.8707671
Osteoarthritis      2.1053998
Pneumonia           0.0216149
NoPneumonia         0.0022132
[3 0 1 2]
Image path: /content/drive/MyDrive/data/test/evTjk3.jpg
Top 4 results: 
class_id            probability
-------------------------------
Pneumonia           100.0000000
NoOsteoarthritis    0.0000000
NoPneumonia         0.0000000
Osteoarthritis      0.0000000
[2 0 1 3]
Image path: /content/drive/MyDrive/data/test/G7bJ_j.jpg
Top 4 results: 
class_id            probability
-------------------------------
Osteoarthritis      99.9991894
NoOsteoarthritis    0.0006437
NoPneumonia         0.0001634
Pneumonia           0.0000012
[3 2 0 1]
Image path: /content/drive/MyDrive/data/test/f0_Hr_.jpg
Top 4 results: 
class_id            probability
-------------------------------
Pneumonia           99.9999166
Osteoarthritis      0.0000566
NoOsteoarthritis    0.0000213
NoPneumonia         0.0000000
[3 1 0 2]